In [42]:
# Familiar imports
import numpy as np
import pandas as pd

# For ordinal encoding categorical variables, splitting data
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

# For training model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
from lightgbm import LGBMRegressor
import lightgbm as lgb
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

from sklearn.linear_model import LinearRegression

#for analysis
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

import optuna
import warnings

In [2]:
# Load the training data
train = pd.read_csv("30-days-of-ml/train_folds.csv")
test = pd.read_csv("30-days-of-ml/test.csv")
sample_submission=pd.read_csv("30-days-of-ml/sample_submission.csv")

# Preview the data
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target,kfold
0,1,B,B,B,C,B,B,A,E,C,...,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634,4
1,2,B,B,A,A,B,D,A,F,A,...,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233,0
2,3,A,A,A,C,B,D,A,D,A,...,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351,4
3,4,B,B,A,C,B,D,A,E,C,...,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253,1
4,6,A,A,A,C,B,D,A,E,A,...,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260,0


In [3]:
useful_cols=[column for column in train.columns if column not in ('id', 'target', 'kfold')]

In [4]:
train[useful_cols]

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,B,B,B,C,B,B,A,E,C,N,...,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
1,B,B,A,A,B,D,A,F,A,O,...,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
2,A,A,A,C,B,D,A,D,A,F,...,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846
3,B,B,A,C,B,D,A,E,C,K,...,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
4,A,A,A,C,B,D,A,E,A,N,...,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,B,B,A,A,B,D,A,E,A,I,...,0.307883,0.769792,0.450538,0.934360,1.005077,0.853726,0.422541,1.063463,0.697685,0.506404
299996,A,B,A,C,B,B,A,E,E,F,...,0.736713,0.528056,0.508502,0.358247,0.257825,0.433525,0.301015,0.268447,0.577055,0.823611
299997,B,B,A,C,B,C,A,E,G,F,...,0.277074,0.688747,0.372425,0.364936,0.383224,0.551825,0.661007,0.629606,0.714139,0.245732
299998,A,B,A,C,B,B,A,E,E,I,...,0.805963,0.344404,0.424243,0.382028,0.468819,0.351036,0.288768,0.611169,0.380254,0.332030


In [5]:
 # List of categorical columns
object_cols = [col for col in useful_cols if 'cat' in col]
test=test[useful_cols]


# Random Forest

In [18]:
last_scores=[]
for fold in range(5):
    X_train = train[train.kfold!=fold].reset_index(drop=True)
    X_valid= train[train.kfold==fold].reset_index(drop=True)
    X_test=test.copy()
    
    y_train=X_train.target
    y_valid=X_valid.target
    
    X_train=X_train[useful_cols]
    X_valid=X_valid[useful_cols]
   
    # ordinal-encode categorical columns
    print('Encoding')
    ordinal_encoder = OrdinalEncoder()
    
    X_train[object_cols]=ordinal_encoder.fit_transform(X_train[object_cols])
    X_valid[object_cols]=ordinal_encoder.transform(X_valid[object_cols])
    X_test[object_cols] = ordinal_encoder.transform(X_test[object_cols])
    
    print('Training') 
    model = RandomForestRegressor(random_state=fold, verbose=100)
    model.fit(X_train,y_train)
    predictions=model.predict(X_valid)
    predictions_test=model.predict(X_test)
    last_scores.append(predictions_test)
    print(fold, mean_squared_error(y_valid,predictions,squared=False))

Encoding
Training
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
building tree 1 of 100
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.9s remaining:    0.0s
building tree 2 of 100
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   17.7s remaining:    0.0s
building tree 3 of 100
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   26.6s remaining:    0.0s
building tree 4 of 100
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   35.5s remaining:    0.0s
building tree 5 of 100
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   44.7s remaining:    0.0s
building tree 6 of 100
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:   53.8s remaining:    0.0s
building tree 7 of 100
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:  1.0min remaining:    0.0s
building tree 8 of 100
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  1.2min remaining:    0.0s
building tree 9 of 100
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  

[Parallel(n_jobs=1)]: Done  79 out of  79 | elapsed: 11.9min remaining:    0.0s
building tree 80 of 100
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed: 12.1min remaining:    0.0s
building tree 81 of 100
[Parallel(n_jobs=1)]: Done  81 out of  81 | elapsed: 12.2min remaining:    0.0s
building tree 82 of 100
[Parallel(n_jobs=1)]: Done  82 out of  82 | elapsed: 12.4min remaining:    0.0s
building tree 83 of 100
[Parallel(n_jobs=1)]: Done  83 out of  83 | elapsed: 12.5min remaining:    0.0s
building tree 84 of 100
[Parallel(n_jobs=1)]: Done  84 out of  84 | elapsed: 12.7min remaining:    0.0s
building tree 85 of 100
[Parallel(n_jobs=1)]: Done  85 out of  85 | elapsed: 12.8min remaining:    0.0s
building tree 86 of 100
[Parallel(n_jobs=1)]: Done  86 out of  86 | elapsed: 13.0min remaining:    0.0s
building tree 87 of 100
[Parallel(n_jobs=1)]: Done  87 out of  87 | elapsed: 13.2min remaining:    0.0s
building tree 88 of 100
[Parallel(n_jobs=1)]: Done  88 out of  88 | elapsed: 13.3min rem

[Parallel(n_jobs=1)]: Done  77 out of  77 | elapsed:    3.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  78 out of  78 | elapsed:    3.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  79 out of  79 | elapsed:    3.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    4.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  81 out of  81 | elapsed:    4.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  82 out of  82 | elapsed:    4.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  83 out of  83 | elapsed:    4.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  84 out of  84 | elapsed:    4.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  85 out of  85 | elapsed:    4.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  86 out of  86 | elapsed:    4.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  87 out of  87 | elapsed:    4.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  88 out of  88 | elapsed:    4.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  89 out of  8

[Parallel(n_jobs=1)]: Done  79 out of  79 | elapsed:   13.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:   13.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  81 out of  81 | elapsed:   13.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  82 out of  82 | elapsed:   13.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  83 out of  83 | elapsed:   13.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  84 out of  84 | elapsed:   13.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  85 out of  85 | elapsed:   14.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  86 out of  86 | elapsed:   14.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  87 out of  87 | elapsed:   14.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  88 out of  88 | elapsed:   14.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  89 out of  89 | elapsed:   14.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed:   14.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  91 out of  9

[Parallel(n_jobs=1)]: Done  62 out of  62 | elapsed:  9.4min remaining:    0.0s
building tree 63 of 100
[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:  9.5min remaining:    0.0s
building tree 64 of 100
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:  9.7min remaining:    0.0s
building tree 65 of 100
[Parallel(n_jobs=1)]: Done  65 out of  65 | elapsed:  9.8min remaining:    0.0s
building tree 66 of 100
[Parallel(n_jobs=1)]: Done  66 out of  66 | elapsed: 10.0min remaining:    0.0s
building tree 67 of 100
[Parallel(n_jobs=1)]: Done  67 out of  67 | elapsed: 10.1min remaining:    0.0s
building tree 68 of 100
[Parallel(n_jobs=1)]: Done  68 out of  68 | elapsed: 10.3min remaining:    0.0s
building tree 69 of 100
[Parallel(n_jobs=1)]: Done  69 out of  69 | elapsed: 10.4min remaining:    0.0s
building tree 70 of 100
[Parallel(n_jobs=1)]: Done  70 out of  70 | elapsed: 10.6min remaining:    0.0s
building tree 71 of 100
[Parallel(n_jobs=1)]: Done  71 out of  71 | elapsed: 10.7min rem

...

[Parallel(n_jobs=1)]: Done  44 out of  44 | elapsed:  6.8min remaining:    0.0s
building tree 45 of 100
[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  7.0min remaining:    0.0s
building tree 46 of 100
[Parallel(n_jobs=1)]: Done  46 out of  46 | elapsed:  7.1min remaining:    0.0s
building tree 47 of 100
[Parallel(n_jobs=1)]: Done  47 out of  47 | elapsed:  7.3min remaining:    0.0s
building tree 48 of 100
[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed:  7.4min remaining:    0.0s
building tree 49 of 100
[Parallel(n_jobs=1)]: Done  49 out of  49 | elapsed:  7.6min remaining:    0.0s
building tree 50 of 100
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  7.7min remaining:    0.0s
building tree 51 of 100
[Parallel(n_jobs=1)]: Done  51 out of  51 | elapsed:  7.9min remaining:    0.0s
building tree 52 of 100
[Parallel(n_jobs=1)]: Done  52 out of  52 | elapsed:  8.0min remaining:    0.0s
building tree 53 of 100
[Parallel(n_jobs=1)]: Done  53 out of  53 | elapsed:  8.2min rem

[Parallel(n_jobs=1)]: Done  29 out of  29 | elapsed:    1.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    1.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  31 out of  31 | elapsed:    1.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    1.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  33 out of  33 | elapsed:    1.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  34 out of  34 | elapsed:    1.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  35 out of  35 | elapsed:    1.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:    1.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  37 out of  37 | elapsed:    1.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  38 out of  38 | elapsed:    1.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  39 out of  39 | elapsed:    1.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    1.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  41 out of  4

[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    5.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  33 out of  33 | elapsed:    5.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  34 out of  34 | elapsed:    5.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  35 out of  35 | elapsed:    5.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:    5.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  37 out of  37 | elapsed:    6.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  38 out of  38 | elapsed:    6.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  39 out of  39 | elapsed:    6.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    6.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  41 out of  41 | elapsed:    6.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  42 out of  42 | elapsed:    6.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  43 out of  43 | elapsed:    7.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  44 out of  4

[Parallel(n_jobs=1)]: Done  26 out of  26 | elapsed:  3.9min remaining:    0.0s
building tree 27 of 100
[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  4.1min remaining:    0.0s
building tree 28 of 100
[Parallel(n_jobs=1)]: Done  28 out of  28 | elapsed:  4.2min remaining:    0.0s
building tree 29 of 100
[Parallel(n_jobs=1)]: Done  29 out of  29 | elapsed:  4.4min remaining:    0.0s
building tree 30 of 100
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  4.6min remaining:    0.0s
building tree 31 of 100
[Parallel(n_jobs=1)]: Done  31 out of  31 | elapsed:  4.7min remaining:    0.0s
building tree 32 of 100
[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:  4.9min remaining:    0.0s
building tree 33 of 100
[Parallel(n_jobs=1)]: Done  33 out of  33 | elapsed:  5.0min remaining:    0.0s
building tree 34 of 100
[Parallel(n_jobs=1)]: Done  34 out of  34 | elapsed:  5.2min remaining:    0.0s
building tree 35 of 100
[Parallel(n_jobs=1)]: Done  35 out of  35 | elapsed:  5.3min rem

KeyboardInterrupt: 

# Light GBMTunned

In [6]:
lgbm_parameters = {
   'metric': 'rmse',
    'learning_rate':.008,
    'n_estimators': 50000,
    'reg_alpha': 9.56,
    'reg_lambda': 9.35,
    'colsample_bytree': 0.21497646795452627,
    'subsample': 0.7582562557431147,
    'learning_rate': 0.009985133666265425,
    'max_depth': 77,
    'num_leaves': 77,
    'min_child_samples': 290,
    'max_bin': 772,
    'cat_l2': 17,
    'cat_smoth':96,
    'seed': 2
}


In [8]:
last_scores=[]
rmse_scores=[]
for fold in range(5):
    X_train = train[train.kfold!=fold].reset_index(drop=True)
    X_valid= train[train.kfold==fold].reset_index(drop=True)
    X_test=test.copy()
    
    y_train=X_train.target
    y_valid=X_valid.target
    
    X_train=X_train[useful_cols]
    X_valid=X_valid[useful_cols]
   
    # ordinal-encode categorical columns
    print('Encoding')
    ordinal_encoder = OrdinalEncoder()
    
    X_train[object_cols]=ordinal_encoder.fit_transform(X_train[object_cols])
    X_valid[object_cols]=ordinal_encoder.transform(X_valid[object_cols])
    X_test[object_cols] = ordinal_encoder.transform(X_test[object_cols])
    
    print('Training') 
    model =LGBMRegressor(**lgbm_parameters)
    model.fit(X_train, y_train, eval_set = ((X_valid,y_valid)), early_stopping_rounds = 200,verbose=1000,
              categorical_feature=object_cols)
    predictions=model.predict(X_valid)
    predictions_test=model.predict(X_test)
    last_scores.append(predictions_test)
    rmse=mean_squared_error(y_valid,predictions,squared=False)
    print(fold, rmse)
    rmse_scores.append(rmse)

Encoding
Training
[LightGBM] [Warning] Unknown parameter: cat_smoth


C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1705: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9']
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.71917
[2000]	valid_0's rmse: 0.715942
[3000]	valid_0's rmse: 0.715338
Early stopping, best iteration is:
[3299]	valid_0's rmse: 0.715289
0 0.7152890254212041
Encoding
Training
[LightGBM] [Warning] Unknown parameter: cat_smoth


C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1705: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9']
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.727117
[2000]	valid_0's rmse: 0.723855
[3000]	valid_0's rmse: 0.723238
Early stopping, best iteration is:
[3183]	valid_0's rmse: 0.723219
1 0.7232186600094757
Encoding
Training
[LightGBM] [Warning] Unknown parameter: cat_smoth


...

Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.722435
[2000]	valid_0's rmse: 0.719259
[3000]	valid_0's rmse: 0.718583
[4000]	valid_0's rmse: 0.718441
Early stopping, best iteration is:
[4008]	valid_0's rmse: 0.71844
2 0.7184400074692084
Encoding
Training
[LightGBM] [Warning] Unknown parameter: cat_smoth


C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1705: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9']
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.72146
[2000]	valid_0's rmse: 0.718539
[3000]	valid_0's rmse: 0.718017
Early stopping, best iteration is:
[3687]	valid_0's rmse: 0.717952
3 0.7179519662088705
Encoding
Training
[LightGBM] [Warning] Unknown parameter: cat_smoth


C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1705: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9']
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.717228
[2000]	valid_0's rmse: 0.714087
[3000]	valid_0's rmse: 0.713509
Early stopping, best iteration is:
[3376]	valid_0's rmse: 0.713436
4 0.7134359170022957


In [ ]:
print(np.mean(rmse_scores), np.std(rmse_scores))
#0.71766711522221088

In [10]:
np.column_stack(last_scores).shape
#the five arrays are for every fold

(200000, 5)

In [22]:
last_scores

[array([8.05848806, 8.36995011, 8.42735571, ..., 8.49073457, 8.18897189,
        8.04246291]),
 array([8.03001228, 8.28199395, 8.40922208, ..., 8.46566595, 8.13418058,
        7.90576582]),
 array([7.9932608 , 8.33373381, 8.35507225, ..., 8.4573602 , 8.06991463,
        7.9837623 ]),
 array([8.01472017, 8.39058241, 8.35813248, ..., 8.45091867, 8.08682882,
        7.86114897]),
 array([8.07038851, 8.35901012, 8.35926018, ..., 8.49227046, 8.13488447,
        7.98448404])]

In [16]:
np.mean(np.column_stack(last_scores),axis=1)

array([8.03337397, 8.34705408, 8.38180854, ..., 8.47138997, 8.12295608,
       7.95552481])

In [12]:
preds=np.mean(np.column_stack(last_scores),axis=1)

In [19]:
sample_submission.target=preds
sample_submission

,id,target
0,0,8.033374
1,5,8.347054
2,15,8.381809
3,16,8.460419
4,17,8.145011
...,...,...
199995,499987,8.058993
199996,499990,8.447804
199997,499991,8.471390
199998,499994,8.122956


In [20]:
sample_submission.to_csv('submission_lgb1.csv',index=False)

# LGBM Standarization 

In [24]:
useful_cols=[column for column in train.columns if column not in ('id', 'target', 'kfold')]

In [25]:
 # List of categorical columns
object_cols = [col for col in useful_cols if 'cat' in col]
numerical_cols = [col for col in useful_cols if 'cont' in col]
test=test[useful_cols]

In [27]:
lgbm_parameters = {
   'metric': 'rmse',
    'learning_rate':.008,
    'n_estimators': 50000,
    'reg_alpha': 9.56,
    'reg_lambda': 9.35,
    'colsample_bytree': 0.21497646795452627,
    'subsample': 0.7582562557431147,
    'learning_rate': 0.009985133666265425,
    'max_depth': 77,
    'num_leaves': 77,
    'min_child_samples': 290,
    'max_bin': 772,
    'cat_l2': 17,
    'cat_smoth':96,
    'seed': 2
}

In [28]:
last_scores=[]
rmse_scores=[]
for fold in range(5):
    X_train = train[train.kfold!=fold].reset_index(drop=True)
    X_valid= train[train.kfold==fold].reset_index(drop=True)
    X_test=test.copy()
    
    y_train=X_train.target
    y_valid=X_valid.target
    
    X_train=X_train[useful_cols]
    X_valid=X_valid[useful_cols]
   
    # ordinal-encode categorical columns
    print('Encoding')
    ordinal_encoder = OrdinalEncoder()
    #for object cols
    X_train[object_cols]=ordinal_encoder.fit_transform(X_train[object_cols])
    X_valid[object_cols]=ordinal_encoder.transform(X_valid[object_cols])
    X_test[object_cols] = ordinal_encoder.transform(X_test[object_cols])
    
    #for numerical cols
    scaler =StandardScaler()
    X_train[numerical_cols]=scaler.fit_transform(X_train[numerical_cols])
    X_valid[numerical_cols]=scaler.transform(X_valid[numerical_cols])
    X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])
    
    print('Training') 
    model =LGBMRegressor(**lgbm_parameters)
    model.fit(X_train, y_train, eval_set = ((X_valid,y_valid)), early_stopping_rounds = 200,verbose=1000,
              categorical_feature=object_cols)
    predictions=model.predict(X_valid)
    predictions_test=model.predict(X_test)
    last_scores.append(predictions_test)
    rmse=mean_squared_error(y_valid,predictions,squared=False)
    print(fold, rmse)
    rmse_scores.append(rmse)

Encoding
Training
[LightGBM] [Warning] Unknown parameter: cat_smoth


C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1705: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9']
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.719228
[2000]	valid_0's rmse: 0.715967
[3000]	valid_0's rmse: 0.71534
Early stopping, best iteration is:
[3752]	valid_0's rmse: 0.715285
0 0.7152850409432048
Encoding
Training
[LightGBM] [Warning] Unknown parameter: cat_smoth


C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1705: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9']
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.727106
[2000]	valid_0's rmse: 0.723789
[3000]	valid_0's rmse: 0.72319
Early stopping, best iteration is:
[3164]	valid_0's rmse: 0.723168
1 0.7231680926003768
Encoding
Training
[LightGBM] [Warning] Unknown parameter: cat_smoth


...

Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.722376
[2000]	valid_0's rmse: 0.71917
[3000]	valid_0's rmse: 0.718503
Early stopping, best iteration is:
[3741]	valid_0's rmse: 0.718363
2 0.718363133332485
Encoding
Training
[LightGBM] [Warning] Unknown parameter: cat_smoth


C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1705: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9']
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.721429
[2000]	valid_0's rmse: 0.718527
[3000]	valid_0's rmse: 0.717976
Early stopping, best iteration is:
[3435]	valid_0's rmse: 0.717921
3 0.7179209441906655
Encoding
Training
[LightGBM] [Warning] Unknown parameter: cat_smoth


C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1705: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9']
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.717252
[2000]	valid_0's rmse: 0.714073
[3000]	valid_0's rmse: 0.713494
Early stopping, best iteration is:
[3191]	valid_0's rmse: 0.713465
4 0.7134645341782053


In [29]:
print(np.mean(rmse_scores), np.std(rmse_scores))

0.7176403490489875 0.0032906536017071808


# LGBM Standarization and hot encoder

In [30]:
useful_cols=[column for column in train.columns if column not in ('id', 'target', 'kfold')]
 # List of categorical columns
object_cols = [col for col in useful_cols if 'cat' in col]
numerical_cols = [col for col in useful_cols if 'cont' in col]
test=test[useful_cols]

In [31]:
lgbm_parameters = {
   'metric': 'rmse',
    'learning_rate':.008,
    'n_estimators': 50000,
    'reg_alpha': 9.56,
    'reg_lambda': 9.35,
    'colsample_bytree': 0.21497646795452627,
    'subsample': 0.7582562557431147,
    'learning_rate': 0.009985133666265425,
    'max_depth': 77,
    'num_leaves': 77,
    'min_child_samples': 290,
    'max_bin': 772,
    'cat_l2': 17,
    'cat_smoth':96,
    'seed': 42
}

In [48]:
last_scores=[]
rmse_scores=[]
for fold in range(5):
    X_train = train[train.kfold!=fold].reset_index(drop=True)
    X_valid= train[train.kfold==fold].reset_index(drop=True)
    X_test=test.copy()
    
    y_train=X_train.target
    y_valid=X_valid.target
    
    X_train=X_train[useful_cols]
    X_valid=X_valid[useful_cols]
   
    # ordinal-encode categorical columns
    print('Encoding')
    ohe = preprocessing.OneHotEncoder(sparse=False, handle_unknown='ignore')
    #for object cols
    X_train_ohe=ohe.fit_transform(X_train[object_cols])
    X_valid_ohe=ohe.transform(X_valid[object_cols])
    X_test_ohe = ohe.transform(X_test[object_cols])
    
    X_train_ohe=pd.DataFrame(X_train_ohe, columns=[f"ohe_{i}" for i in range(X_train_ohe.shape[1])])
    X_valid_ohe=pd.DataFrame(X_valid_ohe, columns=[f"ohe_{i}" for i in range(X_valid_ohe.shape[1])])
    X_test_ohe=pd.DataFrame(X_test_ohe, columns=[f"ohe_{i}" for i in range(X_test_ohe.shape[1])])
    
    X_train=pd.concat([X_train,X_train_ohe], axis=1)
    X_valid=pd.concat([X_valid,X_valid_ohe], axis=1)
    X_test=pd.concat([X_test,X_test_ohe], axis=1)
    
    # Remove categorical columns (will replace with one-hot encoding)
    X_train = X_train.drop(object_cols, axis=1)
    X_valid = X_valid.drop(object_cols,axis=1)
    X_test=X_test.drop(object_cols,axis=1)
    #for numerical cols
    scaler =StandardScaler()
    X_train[numerical_cols]=scaler.fit_transform(X_train[numerical_cols])
    X_valid[numerical_cols]=scaler.transform(X_valid[numerical_cols])
    X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])
    
    print('Training') 
    model =LGBMRegressor(**lgbm_parameters)
    model.fit(X_train, y_train, eval_set = ((X_valid,y_valid)), early_stopping_rounds = 200,verbose=1000)
    predictions=model.predict(X_valid)
    predictions_test=model.predict(X_test)
    last_scores.append(predictions_test)
    rmse=mean_squared_error(y_valid,predictions,squared=False)
    print(fold, rmse)
    rmse_scores.append(rmse)
    

Encoding
Training
[LightGBM] [Warning] Unknown parameter: cat_smoth
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.718989
[2000]	valid_0's rmse: 0.715893
[3000]	valid_0's rmse: 0.715266
Early stopping, best iteration is:
[3706]	valid_0's rmse: 0.715174
0 0.7151735211244796
Encoding
Training
[LightGBM] [Warning] Unknown parameter: cat_smoth
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.726796
[2000]	valid_0's rmse: 0.723509
[3000]	valid_0's rmse: 0.722881
Early stopping, best iteration is:
[3743]	valid_0's rmse: 0.722806
1 0.7228064131919003
Encoding
Training
[LightGBM] [Warning] Unknown parameter: cat_smoth
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.722138
[2000]	valid_0's rmse: 0.71897
[3000]	valid_0's rmse: 0.71829
Early stopping, best iteration is:
[3794]	valid_0's rmse: 0.718173
2 0.7181730818268607
Encoding
Training
[LightGBM] [Warning] Unknown parameter: ca

In [49]:
print(np.mean(rmse_scores), np.std(rmse_scores))

0.7174741924285338 0.0032032586978842557


In [5]:
useful_cols=[column for column in train.columns if column not in ('id', 'target', 'kfold')]
 # List of categorical columns
object_cols = [col for col in useful_cols if 'cat' in col]
numerical_cols = [col for col in useful_cols if 'cont' in col]
test=test[useful_cols]

In [6]:
def run(trial):
    #last_scores=[]
    #rmse_scores=[]
    fold=0
    learning_rate=trial.suggest_float('learning_rate', .008,.25, log=True)
    reg_lambda=trial.suggest_loguniform('reg_lambda', 1e-8, 100.0)
    reg_alpha=trial.suggest_loguniform('reg_alpha', 1e-8, 100.0)
    subsample=trial.suggest_float('subsamble', .1, 1.0)
    colsample_bytree=trial.suggest_float('colsample_bytree', .1, 1.0)
    max_depth=trial.suggest_int('max_depth',10,77)
    num_leaves=trial.suggest_int('num_leaves',10,77)
    min_child_samples=trial.suggest_int('min_child_samples',10,100)
        
    X_train = train[train.kfold!=fold].reset_index(drop=True)
    X_valid= train[train.kfold==fold].reset_index(drop=True)
    #X_test=test.copy()

    y_train=X_train.target
    y_valid=X_valid.target

    X_train=X_train[useful_cols]
    X_valid=X_valid[useful_cols]

        # ordinal-encode categorical columns
    print('Encoding')
    ohe = preprocessing.OneHotEncoder(sparse=False, handle_unknown='ignore')
        #for object cols
    X_train_ohe=ohe.fit_transform(X_train[object_cols])
    X_valid_ohe=ohe.transform(X_valid[object_cols])
        #X_test_ohe = ohe.transform(X_test[object_cols])

    X_train_ohe=pd.DataFrame(X_train_ohe, columns=[f"ohe_{i}" for i in range(X_train_ohe.shape[1])])
    X_valid_ohe=pd.DataFrame(X_valid_ohe, columns=[f"ohe_{i}" for i in range(X_valid_ohe.shape[1])])
        #X_test_ohe=pd.DataFrame(X_test_ohe, columns=[f"ohe_{i}" for i in range(X_test_ohe.shape[1])])

    X_train=pd.concat([X_train,X_train_ohe], axis=1)
    X_valid=pd.concat([X_valid,X_valid_ohe], axis=1)
        #X_test=pd.concat([X_test,X_test_ohe], axis=1)

        # Remove categorical columns (will replace with one-hot encoding)
    X_train = X_train.drop(object_cols, axis=1)
    X_valid = X_valid.drop(object_cols,axis=1)
        #X_test=X_test.drop(object_cols,axis=1)
        #for numerical cols
    scaler =StandardScaler()
    X_train[numerical_cols]=scaler.fit_transform(X_train[numerical_cols])
    X_valid[numerical_cols]=scaler.transform(X_valid[numerical_cols])
        #X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

    print('Training') 
    model =LGBMRegressor(random_state=42,
                            learning_rate=learning_rate, 
                            reg_lambda=reg_lambda,
                            reg_alpha=reg_alpha,
                            subsample=subsample,
                            colsample_bytree=colsample_bytree,
                            max_depth=max_depth,
                            num_leaves=num_leaves,
                            min_child_samples=min_child_samples,
                             n_estimators=7000
                            )
    model.fit(X_train, y_train, eval_set = ((X_valid,y_valid)), early_stopping_rounds = 200,verbose=1000)
    predictions=model.predict(X_valid)
        #predictions_test=model.predict(X_test)
        #last_scores.append(predictions_test)
    rmse=mean_squared_error(y_valid,predictions,squared=False)
    print(fold, rmse)
    return rmse   
    

In [7]:
study = optuna.create_study(direction='minimize')
study.optimize(run, n_trials=100)
study.best_params

[I 2021-08-31 05:40:09,169] A new study created in memory with name: no-name-c7451d32-c7d3-4c69-9264-0261fbd98c59


Encoding
Training
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[265]	valid_0's l2: 0.516758


[I 2021-08-31 05:40:18,334] Trial 0 finished with value: 0.7188586717929577 and parameters: {'learning_rate': 0.19185444850119765, 'reg_lambda': 7.254606589829645, 'reg_alpha': 5.325796952954004e-08, 'subsamble': 0.5165904606780625, 'colsample_bytree': 0.7880680141138762, 'max_depth': 53, 'num_leaves': 17, 'min_child_samples': 52}. Best is trial 0 with value: 0.7188586717929577.


0 0.7188586717929577
Encoding
Training
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[581]	valid_0's l2: 0.515577


[I 2021-08-31 05:40:34,291] Trial 1 finished with value: 0.7180365596310633 and parameters: {'learning_rate': 0.09940899072455849, 'reg_lambda': 6.672585616147851e-07, 'reg_alpha': 9.688706189916708, 'subsamble': 0.4691578019300532, 'colsample_bytree': 0.7441597659589874, 'max_depth': 44, 'num_leaves': 23, 'min_child_samples': 89}. Best is trial 1 with value: 0.7180365596310633.


...

[I 2021-08-31 08:02:09,748] Trial 98 finished with value: 0.715898293835209 and parameters: {'learning_rate': 0.02415153450969747, 'reg_lambda': 2.9051904393725803, 'reg_alpha': 3.1110322523775897, 'subsamble': 0.756664516131287, 'colsample_bytree': 0.13511493321954277, 'max_depth': 45, 'num_leaves': 30, 'min_child_samples': 59}. Best is trial 74 with value: 0.7157022024939034.


0 0.715898293835209
Encoding
Training
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's l2: 0.517925
[2000]	valid_0's l2: 0.513953
[3000]	valid_0's l2: 0.513117
[4000]	valid_0's l2: 0.51281
Early stopping, best iteration is:
[4711]	valid_0's l2: 0.512679


[I 2021-08-31 08:04:06,702] Trial 99 finished with value: 0.7160161396703324 and parameters: {'learning_rate': 0.011911441300670662, 'reg_lambda': 0.20917686602976024, 'reg_alpha': 6.698260942483536, 'subsamble': 0.8561663772509371, 'colsample_bytree': 0.19778420913805123, 'max_depth': 55, 'num_leaves': 42, 'min_child_samples': 66}. Best is trial 74 with value: 0.7157022024939034.


0 0.7160161396703324


{'learning_rate': 0.017617905594164103,
 'reg_lambda': 0.2971710372778072,
 'reg_alpha': 0.4754381939584199,
 'subsamble': 0.9808300822318052,
 'colsample_bytree': 0.10159075122483158,
 'max_depth': 27,
 'num_leaves': 27,
 'min_child_samples': 76}

In [24]:
lgbm_parameters = {
   'metric': 'rmse',
   'learning_rate': 0.017617905594164103,
    'n_estimators': 50000,
     'reg_alpha': 0.4754381939584199,
    'reg_lambda': 0.2971710372778072,
    'colsample_bytree': 0.10159075122483158,
     'subsample': 0.9808300822318052,
    'max_depth': 27,
    'num_leaves': 27,
    'min_child_samples': 76,
    'max_bin': 772,
    'cat_l2': 17,
    'cat_smoth':96,
    'seed': 42
}

In [18]:
useful_cols=[column for column in train.columns if column not in ('id', 'target', 'kfold')]
 # List of categorical columns
object_cols = [col for col in useful_cols if 'cat' in col]
numerical_cols = [col for col in useful_cols if 'cont' in col]
test=test[useful_cols]

final_test_predictions=[]
final_valid_predictions={}

last_scores=[]
rmse_scores=[]
for fold in range(5):
    X_train = train[train.kfold!=fold].reset_index(drop=True)
    X_valid= train[train.kfold==fold].reset_index(drop=True)
    X_test=test.copy()
    
    valid_ids=X_valid.id.values.tolist()
    
    y_train=X_train.target
    y_valid=X_valid.target
    
    X_train=X_train[useful_cols]
    X_valid=X_valid[useful_cols]
   
   # ordinal-encode categorical columns
    print('Encoding')
    ohe = preprocessing.OneHotEncoder(sparse=False, handle_unknown='ignore')
    #for object cols
    X_train_ohe=ohe.fit_transform(X_train[object_cols])
    X_valid_ohe=ohe.transform(X_valid[object_cols])
    X_test_ohe = ohe.transform(X_test[object_cols])
    
    X_train_ohe=pd.DataFrame(X_train_ohe, columns=[f"ohe_{i}" for i in range(X_train_ohe.shape[1])])
    X_valid_ohe=pd.DataFrame(X_valid_ohe, columns=[f"ohe_{i}" for i in range(X_valid_ohe.shape[1])])
    X_test_ohe=pd.DataFrame(X_test_ohe, columns=[f"ohe_{i}" for i in range(X_test_ohe.shape[1])])
    
    X_train=pd.concat([X_train,X_train_ohe], axis=1)
    X_valid=pd.concat([X_valid,X_valid_ohe], axis=1)
    X_test=pd.concat([X_test,X_test_ohe], axis=1)
    
    # Remove categorical columns (will replace with one-hot encoding)
    X_train = X_train.drop(object_cols, axis=1)
    X_valid = X_valid.drop(object_cols,axis=1)
    X_test=X_test.drop(object_cols,axis=1)
    
    
    #for numerical cols
    scaler =StandardScaler()
    X_train[numerical_cols]=scaler.fit_transform(X_train[numerical_cols])
    X_valid[numerical_cols]=scaler.transform(X_valid[numerical_cols])
    X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])
    
    print('Training') 
    model =LGBMRegressor(**lgbm_parameters)
    model.fit(X_train, y_train, eval_set = ((X_valid,y_valid)), early_stopping_rounds = 500,verbose=1000)
    predictions=model.predict(X_valid)
    predictions_test=model.predict(X_test)
    final_test_predictions.append(predictions_test)
    final_valid_predictions.update(dict(zip(valid_ids,predictions)))
    
    rmse=mean_squared_error(y_valid,predictions,squared=False)
    print(fold, rmse)
    rmse_scores.append(rmse)
    

Encoding
Training
[LightGBM] [Warning] Unknown parameter: cat_smoth
Training until validation scores don't improve for 500 rounds
[1000]	valid_0's rmse: 0.718293
[2000]	valid_0's rmse: 0.71575
[3000]	valid_0's rmse: 0.715166
[4000]	valid_0's rmse: 0.714951
[5000]	valid_0's rmse: 0.714905
Early stopping, best iteration is:
[4799]	valid_0's rmse: 0.714891
0 0.7148905224844296
Encoding
Training
[LightGBM] [Warning] Unknown parameter: cat_smoth
Training until validation scores don't improve for 500 rounds
[1000]	valid_0's rmse: 0.726042
[2000]	valid_0's rmse: 0.723235
[3000]	valid_0's rmse: 0.722656
[4000]	valid_0's rmse: 0.722418
[5000]	valid_0's rmse: 0.722395
Early stopping, best iteration is:
[4759]	valid_0's rmse: 0.722377
1 0.7223767670022336
Encoding
Training
[LightGBM] [Warning] Unknown parameter: cat_smoth
Training until validation scores don't improve for 500 rounds
[1000]	valid_0's rmse: 0.721522
[2000]	valid_0's rmse: 0.718926
[3000]	valid_0's rmse: 0.718358
[4000]	valid_0's rm

In [19]:
print(np.mean(rmse_scores), np.std(rmse_scores))

0.7172132968073746 0.0031972227229733487


In [20]:
final_valid_predictions= pd.DataFrame.from_dict(final_valid_predictions, orient='index').reset_index()
final_valid_predictions.columns=['id', 'pred_1']
final_valid_predictions.to_csv('train_pred_1.csv', index=False)

In [21]:
final_valid_predictions

,id,pred_1
0,2,8.355080
1,6,8.256156
2,8,8.460447
3,10,8.445550
4,18,8.189734
...,...,...
299995,499975,8.194221
299996,499976,8.611674
299997,499993,8.357469
299998,499996,7.802166


In [23]:
sample_submission.target=np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns=['id', 'pred_1']
sample_submission.to_csv('test_pred_1.csv', index=False)

In [25]:
useful_cols=[column for column in train.columns if column not in ('id', 'target', 'kfold')]
 # List of categorical columns
object_cols = [col for col in useful_cols if 'cat' in col]
numerical_cols = [col for col in useful_cols if 'cont' in col]
test=test[useful_cols]

final_test_predictions=[]
final_valid_predictions={}

last_scores=[]
rmse_scores=[]
for fold in range(5):
    X_train = train[train.kfold!=fold].reset_index(drop=True)
    X_valid= train[train.kfold==fold].reset_index(drop=True)
    X_test=test.copy()
    
    valid_ids=X_valid.id.values.tolist()
    
    y_train=X_train.target
    y_valid=X_valid.target
    
    X_train=X_train[useful_cols]
    X_valid=X_valid[useful_cols]
   
    # ordinal-encode categorical columns
    print('Encoding')
    ordinal_encoder = OrdinalEncoder()
    #for object cols
    X_train[object_cols]=ordinal_encoder.fit_transform(X_train[object_cols])
    X_valid[object_cols]=ordinal_encoder.transform(X_valid[object_cols])
    X_test[object_cols] = ordinal_encoder.transform(X_test[object_cols])
    
    #for numerical cols
    scaler =StandardScaler()
    X_train[numerical_cols]=scaler.fit_transform(X_train[numerical_cols])
    X_valid[numerical_cols]=scaler.transform(X_valid[numerical_cols])
    X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])
    
    print('Training') 
    model =LGBMRegressor(**lgbm_parameters)
    model.fit(X_train, y_train, eval_set = ((X_valid,y_valid)), early_stopping_rounds = 200,verbose=1000,
              categorical_feature=object_cols)
    predictions=model.predict(X_valid)
    predictions_test=model.predict(X_test)
    final_test_predictions.append(predictions_test)
    final_valid_predictions.update(dict(zip(valid_ids,predictions)))
    
    
    rmse=mean_squared_error(y_valid,predictions,squared=False)
    print(fold, rmse)
    rmse_scores.append(rmse)

Encoding
Training
[LightGBM] [Warning] Unknown parameter: cat_smoth


C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1705: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9']
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.721292
[2000]	valid_0's rmse: 0.716902
[3000]	valid_0's rmse: 0.71569
[4000]	valid_0's rmse: 0.715257
[5000]	valid_0's rmse: 0.71512
Early stopping, best iteration is:
[4826]	valid_0's rmse: 0.715117
0 0.7151167023414063
Encoding
Training
[LightGBM] [Warning] Unknown parameter: cat_smoth


C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1705: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9']
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.728824
[2000]	valid_0's rmse: 0.724469
[3000]	valid_0's rmse: 0.723224
[4000]	valid_0's rmse: 0.722791
[5000]	valid_0's rmse: 0.722688
Early stopping, best iteration is:
[4816]	valid_0's rmse: 0.722665
1 0.7226651859902857
Encoding
Training
[LightGBM] [Warning] Unknown parameter: cat_smoth


...

Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.724001
[2000]	valid_0's rmse: 0.719829
[3000]	valid_0's rmse: 0.718739
[4000]	valid_0's rmse: 0.718375
Early stopping, best iteration is:
[4044]	valid_0's rmse: 0.718361
2 0.7183612417693777
Encoding
Training
[LightGBM] [Warning] Unknown parameter: cat_smoth


C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1705: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9']
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.723782
[2000]	valid_0's rmse: 0.71968
[3000]	valid_0's rmse: 0.718605
[4000]	valid_0's rmse: 0.71826
Early stopping, best iteration is:
[4563]	valid_0's rmse: 0.71821
3 0.7182098291206642
Encoding
Training
[LightGBM] [Warning] Unknown parameter: cat_smoth


C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1705: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9']
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\celes\anaconda3\lib\site-packages\lightgbm\basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.719376
[2000]	valid_0's rmse: 0.715138
[3000]	valid_0's rmse: 0.714004
[4000]	valid_0's rmse: 0.713585
Early stopping, best iteration is:
[4468]	valid_0's rmse: 0.713553
4 0.7135533994061092


In [26]:
print(np.mean(rmse_scores), np.std(rmse_scores))

0.7175812717255686 0.003135202691705293


In [29]:
final_valid_predictions= pd.DataFrame.from_dict(final_valid_predictions, orient='index').reset_index()
final_valid_predictions.columns=['id', 'pred_2']
final_valid_predictions.to_csv('train_pred_2.csv', index=False)


TypeError: 'numpy.ndarray' object is not callable

In [30]:
final_valid_predictions

,id,pred_2
0,2,8.382772
1,6,8.218328
2,8,8.433875
3,10,8.355328
4,18,8.188500
...,...,...
299995,499975,8.225013
299996,499976,8.613246
299997,499993,8.362486
299998,499996,7.768921


In [28]:
sample_submission.target=np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns=['id', 'pred_2']
sample_submission.to_csv('test_pred_2.csv', index=False)

<ipython-input-28-e20eeb88c83a>:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  sample_submission.target=np.mean(np.column_stack(final_test_predictions), axis=1)


In [33]:


useful_cols=[column for column in train.columns if column not in ('id', 'target', 'kfold')]
 # List of categorical columns
object_cols = [col for col in useful_cols if 'cat' in col]
numerical_cols = [col for col in useful_cols if 'cont' in col]
test=test[useful_cols]

final_test_predictions=[]
final_valid_predictions={}

last_scores=[]
rmse_scores=[]
for fold in range(5):
    X_train = train[train.kfold!=fold].reset_index(drop=True)
    X_valid= train[train.kfold==fold].reset_index(drop=True)
    X_test=test.copy()
    
    valid_ids=X_valid.id.values.tolist()
    
    y_train=X_train.target
    y_valid=X_valid.target
    
    X_train=X_train[useful_cols]
    X_valid=X_valid[useful_cols]
   
    # ordinal-encode categorical columns
    print('Encoding')
    ordinal_encoder = OrdinalEncoder()
    #for object cols
    X_train[object_cols]=ordinal_encoder.fit_transform(X_train[object_cols])
    X_valid[object_cols]=ordinal_encoder.transform(X_valid[object_cols])
    X_test[object_cols] = ordinal_encoder.transform(X_test[object_cols])
    
    #for numerical cols
    scaler =StandardScaler()
    X_train[numerical_cols]=scaler.fit_transform(X_train[numerical_cols])
    X_valid[numerical_cols]=scaler.transform(X_valid[numerical_cols])
    X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])
    
    print('Training') 
     
    model = XGBRegressor(n_estimators=20000, random_state=42, learning_rate=.02, max_depth=5, tree_method='hist')
    model.fit(X_train, y_train, eval_set = [(X_valid,y_valid)], early_stopping_rounds = 400,verbose=1000)
    
    predictions=model.predict(X_valid)
    predictions_test=model.predict(X_test)
    final_test_predictions.append(predictions_test)
    final_valid_predictions.update(dict(zip(valid_ids,predictions)))
    
    
    rmse=mean_squared_error(y_valid,predictions,squared=False)
    print(fold, rmse)
    rmse_scores.append(rmse)

Encoding
Training
[0]	validation_0-rmse:7.62102
[1000]	validation_0-rmse:0.72091
[2000]	validation_0-rmse:0.71862
[3000]	validation_0-rmse:0.71825
[3524]	validation_0-rmse:0.71831
0 0.7182326233845375
Encoding
Training
[0]	validation_0-rmse:7.62425
[1000]	validation_0-rmse:0.72858
[2000]	validation_0-rmse:0.72626
[3000]	validation_0-rmse:0.72588
[3576]	validation_0-rmse:0.72593
1 0.7258669458210288
Encoding
Training
[0]	validation_0-rmse:7.62033
[1000]	validation_0-rmse:0.72427
[2000]	validation_0-rmse:0.72206
[3000]	validation_0-rmse:0.72168
[3157]	validation_0-rmse:0.72176
2 0.7216353173258985
Encoding
Training
[0]	validation_0-rmse:7.62672
[1000]	validation_0-rmse:0.72343
[2000]	validation_0-rmse:0.72138
[3000]	validation_0-rmse:0.72100
[3271]	validation_0-rmse:0.72104
3 0.720987603644735
Encoding
Training
[0]	validation_0-rmse:7.62652
[1000]	validation_0-rmse:0.71879
[2000]	validation_0-rmse:0.71663
[3000]	validation_0-rmse:0.71613
[3447]	validation_0-rmse:0.71621
4 0.7161064903577

In [34]:
print(np.mean(rmse_scores), np.std(rmse_scores))

0.7205657961067833 0.003309171746692743


In [35]:
final_valid_predictions= pd.DataFrame.from_dict(final_valid_predictions, orient='index').reset_index()
final_valid_predictions.columns=['id', 'pred_3']
final_valid_predictions.to_csv('train_pred_3.csv', index=False)

In [36]:
sample_submission.target=np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns=['id', 'pred_3']
sample_submission.to_csv('test_pred_3.csv', index=False)

In [38]:
train = pd.read_csv("30-days-of-ml/train_folds.csv")
test = pd.read_csv("30-days-of-ml/test.csv")
sample_submission=pd.read_csv("30-days-of-ml/sample_submission.csv")

df1 = pd.read_csv("train_pred_1.csv")
df2 = pd.read_csv("train_pred_2.csv")
df3 = pd.read_csv("train_pred_3.csv")

df_test1 = pd.read_csv("test_pred_1.csv")
df_test2 = pd.read_csv("test_pred_2.csv")
df_test3 = pd.read_csv("test_pred_3.csv")

train = train.merge(df1, on="id", how="left")
train = train.merge(df2, on="id", how="left")
train = train.merge(df3, on="id", how="left")

test = test.merge(df_test1, on="id", how="left")
test = test.merge(df_test2, on="id", how="left")
test = test.merge(df_test3, on="id", how="left")

train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont9,cont10,cont11,cont12,cont13,target,kfold,pred_1,pred_2,pred_3
0,1,B,B,B,C,B,B,A,E,C,...,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634,4,8.507232,8.503018,8.544546
1,2,B,B,A,A,B,D,A,F,A,...,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233,0,8.355080,8.382772,8.348386
2,3,A,A,A,C,B,D,A,D,A,...,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351,4,8.175895,8.241156,8.162363
3,4,B,B,A,C,B,D,A,E,C,...,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253,1,8.393389,8.388439,8.402562
4,6,A,A,A,C,B,D,A,E,A,...,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260,0,8.256156,8.218328,8.234509


In [39]:
lgbm_parameters = {
   'metric': 'rmse',
   'learning_rate': 0.017617905594164103,
    'n_estimators': 50000,
     'reg_alpha': 0.4754381939584199,
    'reg_lambda': 0.2971710372778072,
    'colsample_bytree': 0.10159075122483158,
     'subsample': 0.9808300822318052,
    'max_depth': 27,
    'num_leaves': 27,
    'min_child_samples': 76,
    'max_bin': 772,
    'cat_l2': 17,
    'cat_smoth':96,
    'seed': 42
}

In [44]:
useful_features = ["pred_1", "pred_2", "pred_3"]
test = test[useful_features]

In [45]:
test

,pred_1,pred_2,pred_3
0,8.077633,8.077633,8.077633
1,8.373021,8.373021,8.373021
2,8.407936,8.407936,8.407936
3,8.479666,8.479666,8.479666
4,8.149429,8.149429,8.149429
...,...,...,...
199995,8.041326,8.041326,8.041326
199996,8.461375,8.461375,8.461375
199997,8.458954,8.458954,8.458954
199998,8.181627,8.181627,8.181627


In [50]:
useful_features = ["pred_1", "pred_2", "pred_3"]
test = test[useful_features]

final_predictions = []
scores = []
for fold in range(5):
    X_train = train[train.kfold!=fold].reset_index(drop=True)
    X_valid= train[train.kfold==fold].reset_index(drop=True)
    X_test=test.copy()
    
    
    y_train=X_train.target
    y_valid=X_valid.target
    
    X_train=X_train[useful_features]
    X_valid=X_valid[useful_features]
   
    
    print('Training') 
    model =LGBMRegressor(**lgbm_parameters)
    model.fit(X_train, y_train, eval_set = ((X_valid,y_valid)), early_stopping_rounds = 500,verbose=1000)
    
    #model = LinearRegression()
    #model.fit(X_train, y_train)
    
    predictions=model.predict(X_valid)
    predictions_test=model.predict(X_test)
    final_predictions.append(predictions_test)
    
    rmse=mean_squared_error(y_valid,predictions,squared=False)
    print(fold, rmse)
    scores.append(rmse)
    


Training
[LightGBM] [Warning] Unknown parameter: cat_smoth
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[249]	valid_0's rmse: 0.714677
0 0.7146769663796279
Training
[LightGBM] [Warning] Unknown parameter: cat_smoth
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[256]	valid_0's rmse: 0.722181
1 0.7221807332882377
Training
[LightGBM] [Warning] Unknown parameter: cat_smoth
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[204]	valid_0's rmse: 0.717906
2 0.7179063023388252
Training
[LightGBM] [Warning] Unknown parameter: cat_smoth
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[185]	valid_0's rmse: 0.717637
3 0.7176370420250531
Training
[LightGBM] [Warning] Unknown parameter: cat_smoth
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[221]	valid_

In [49]:
print(np.mean(scores), np.std(scores))

0.7170760635232711 0.0031659183145246596


In [51]:
print(np.mean(scores), np.std(scores))

0.7170509155028089 0.0031825747977556563


In [52]:
sample_submission.target = np.mean(np.column_stack(final_predictions), axis=1)
sample_submission.to_csv("submission_multiple.csv", index=False)